In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import json

import re

import asyncio
from playwright.async_api import async_playwright
import tqdm.asyncio

# Subset to software

In [103]:
df = pd.read_csv("./websites.csv")

In [105]:
len(df)

66342

In [104]:
df.head(2)

,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
0,1543,Add*Ed,http://www.addedrg.ca,['autism' 'intellectual disability' 'teacher t...,"{'city': 'Kingston', 'country': 'canada', 'cou...",Education Management,NaN
1,1952,"Addison AG Tech, LP",http://www.AddisonFund.com,[],"{'city': 'Addison', 'country': 'united states'...",Financial Services,NaN


In [4]:
df.tail(2)

,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
66340,7404336,Wireframe Marketing,https://wireframemarketing.com,['SEO Services' 'Logo Design' 'Branding' 'Soci...,"{'city': 'REXBURG', 'country': 'united states'...",Marketing & Advertising,NaN
66341,7404394,Wirehound Networks,http://www.wirehoundnetworks.com,[],"{'city': 'Richfield', 'country': 'united state...",Computer & Network Security,NaN


In [8]:
df = df[df['Industry'] == 'Computer Software'].reset_index()
len(df)

6548

In [9]:
df.head(2)

,index,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
0,11,5314,Sawa Impact Labs,https://sawa.gallery/,[],"{'city': 'Stanford', 'country': 'united states...",Computer Software,NaN
1,19,5505,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,[],"{'city': 'Ann Arbor', 'country': 'united state...",Computer Software,NaN


In [42]:
def parse_single_quote_json(json_str):
    json_str_fixed = json_str.replace("'", '"').replace("None", "null")
    try:
        return json.loads(json_str_fixed)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None
    
def _get_country(_str):
    x = parse_single_quote_json(_str)
    if x is None:
        return None
    if 'country' not in x.keys():
        return None
    return x['country']
    

In [43]:
df['country'] = df.Address.apply(lambda x: _get_country(x))

Error parsing JSON: Expecting ',' delimiter: line 1 column 19 (char 18)
Error parsing JSON: Expecting ',' delimiter: line 1 column 19 (char 18)
Error parsing JSON: Invalid control character at: line 1 column 18 (char 17)
Error parsing JSON: Expecting ',' delimiter: line 1 column 18 (char 17)
Error parsing JSON: Expecting ',' delimiter: line 1 column 13 (char 12)
Error parsing JSON: Invalid control character at: line 1 column 12 (char 11)
Error parsing JSON: Invalid control character at: line 1 column 81 (char 80)


country is good (was checking if it's full of Chinese/Indian companies)

In [44]:
df.country.value_counts()

country
united states     4734
united kingdom     884
canada             392
australia          254
singapore          172
ireland             79
new zealand         26
Name: count, dtype: int64

In [47]:
df.Website.head(20)

0                                https://sawa.gallery/
1     https://maizepages.umich.edu/organization/openmi
2                                   https://siftmo.com
3                              https://www.aleios.com/
4                                    http://niftys.com
5                                 https://basewell.com
6                                    http://journey.io
7                             https://apple.co/3HcKeLx
8                               https://sqdc.avoyer.ca
9                             http://www.appcoderz.com
10                                  https://appdive.io
11                              https://greenpixie.com
12                                http://tryparade.com
13                       http://www.extendingfamily.us
14                                 https://nixmade.com
15                             https://www.jumproom.io
16                                    https://maple.so
17                         http://www.blueprintgtm.com
18        

In [102]:
df = df[df['country'] == 'united states']
len(df)

4734

# Scrape website

In [59]:
async def scrape_url(url):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch()
            context = browser.new_context()
            page = await browser.new_page()
            
            await page.goto(url)
            
            await page.wait_for_load_state(timeout=20000)
            await page.keyboard.press('PageDown')
            await page.wait_for_load_state(timeout=20000)
            
            href = await page.evaluate('() => document.location.href')
            await page.wait_for_load_state(timeout=20000)
            await page.screenshot(path="screenshot.png", full_page=True)
            
            content = await page.content()
            await browser.close()
            return content
    except Exception as e:
        print(e)
        return None

In [60]:
url = 'https://maizepages.umich.edu/organization/openmi'

In [61]:
content = await scrape_url(url)

/home/krisztian/miniconda3/envs/all/lib/python3.12/inspect.py:479: RuntimeWarning: coroutine 'Browser.new_context' was never awaited
  def isframe(object):


In [84]:
async def limit_parallel(tasks, n=5):
    """
    Run up to n async tasks in parallel.

    :param n: Maximum number of concurrent tasks.
    :param tasks: List of coroutine functions to execute.
    :return: List of results from the tasks.
    """
    semaphore = asyncio.Semaphore(n)

    async def sem_task(task):
        async with semaphore:
            return await task

    # Wrap tasks with semaphore logic
    wrapped_tasks = [sem_task(task) for task in tasks]

    # Run all tasks concurrently
    return await tqdm.asyncio.tqdm.gather(*wrapped_tasks)

In [85]:
urls = df.Website.head(20)

In [95]:
tasks = [scrape_url(u) for u in urls]

In [96]:
results = await limit_parallel(tasks, 10)

  5%|███████████▌                                                                                                                                                                                                                           | 1/20 [00:02<00:45,  2.40s/it]

Page.goto: net::ERR_NAME_NOT_RESOLVED at https://nixmade.com/
Call log:
navigating to "https://nixmade.com/", waiting until "load"



 10%|███████████████████████                                                                                                                                                                                                                | 2/20 [00:02<00:21,  1.20s/it]

Page.goto: net::ERR_CERT_COMMON_NAME_INVALID at http://www.appcoderz.com/
Call log:
navigating to "http://www.appcoderz.com/", waiting until "load"



 45%|███████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                               | 9/20 [00:07<00:09,  1.17it/s]

Page.goto: net::ERR_NAME_NOT_RESOLVED at http://www.extendingfamily.us/
Call log:
navigating to "http://www.extendingfamily.us/", waiting until "load"



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.44it/s]


In [97]:
results[0]

'<!DOCTYPE html><html class="yui3-js-enabled js-focus-visible js flexbox canvas canvastext webgl no-touch hashchange history draganddrop rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms no-csstransforms3d csstransitions video audio svg inlinesvg svgclippaths" data-js-focus-visible="" style=""><div id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important"></div><head>\n\n  <title>Squarespace - Website Expired</title>\n  <meta http-equiv="X-UA-Compatible" content="chrome=1">\n\n  <script src="//assets.squarespace.com/@sqs/polyfiller/1.6/legacy.js" nomodule="" type="text/javascript" crossorigin="anonymous"></script>\n  <script src="//assets.squarespace.com/@sqs/polyfiller/1.6/modern.js" type="text/javascript" crossorigin="anonymous"></script>\n  <script crossorigin="anonymous" src="//assets.squarespace.com/universal/scripts-compressed/extract-css-runt

In [98]:
def _estime_whole_df(secs):
    return secs / 20 * len(df) / 3600

In [100]:
_estime_whole_df(13)

1.1822777777777778

Going to
1. Set of async job for whole list
2. do md conversion
3. do prompt + evals for extracting jobs links / classifying pages / etc

page classifications:
- is it a valid htmls
- does it contains jobs?
    - Yes / list
    - Yes / open apply
    - No / with links
    - No / no links
- extract links

# Do markdown conversion

In [154]:
df2 = pd.read_csv("../data/01_subset_enriched.csv")

In [155]:
df.to_csv("../data/01_subset_enriched.csv", index=False)

In [156]:
df.head(2)

,CompanyName,Website,scrape_status,homepage_content
0,Sawa Impact Labs,https://sawa.gallery/,Success,"<!DOCTYPE html><html class=""yui3-js-enabled js..."
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,"<!DOCTYPE html><html lang=""en""><head>\n <me..."


In [157]:
df.scrape_status.value_counts()

scrape_status
Success        3655
Failed         1071
Not Started       8
Name: count, dtype: int64

In [125]:
failed = df[df.scrape_status == 'Failed']

In [126]:
len(failed)

505

In [127]:
success = df[df.scrape_status== 'Success'].reset_index(drop=True)

In [128]:
len(success)

1722

In [129]:
success.head()

,CompanyName,Website,scrape_status,homepage_content
0,Sawa Impact Labs,https://sawa.gallery/,Success,"<!DOCTYPE html><html class=""yui3-js-enabled js..."
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,"<!DOCTYPE html><html lang=""en""><head>\n <me..."
2,Nifty's,http://niftys.com,Success,"<!DOCTYPE html><html class=""no-js"" lang=""en-US..."
3,Basewell,https://basewell.com,Success,"<!DOCTYPE html><html lang=""en""><head><meta cha..."
4,Journey,http://journey.io,Success,"<!DOCTYPE html><html lang=""en""><head>\n <me..."


In [130]:
from jobsfinder.core import html2md

In [131]:
print(html2md(success.homepage_content.iloc[0]))



Squarespace \- Website Expired


 
Website ExpiredThis account has expired. If you are the site owner, click below to login.


In [132]:
df = success.copy()

In [135]:
len(df)

1722

In [136]:
from tqdm import tqdm

In [137]:
df['md'] = [html2md(x) for x in tqdm(df.homepage_content)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1722/1722 [06:27<00:00,  4.44it/s]


In [138]:
df['html_length'] = df.homepage_content.apply(lambda x: None if x is None else len(x))
df['md_length'] = df.md.apply(lambda x: None if x is None else len(x))

In [139]:
df.html_length.describe()

count    1.722000e+03
mean     2.566247e+05
std      8.206222e+05
min      3.900000e+01
25%      2.496650e+04
50%      8.439500e+04
75%      2.130652e+05
max      1.529246e+07
Name: html_length, dtype: float64

In [140]:
df.md_length.describe()

count      1722.000000
mean       6392.870499
std       11975.955074
min           0.000000
25%        1345.250000
50%        4034.000000
75%        7348.000000
max      261102.000000
Name: md_length, dtype: float64

In [141]:
df.md_length.describe()

count      1722.000000
mean       6392.870499
std       11975.955074
min           0.000000
25%        1345.250000
50%        4034.000000
75%        7348.000000
max      261102.000000
Name: md_length, dtype: float64

In [142]:
df['md2html_ratio'] = df.html_length / df.md_length

In [143]:
df.md2html_ratio.describe()

/home/krisztian/projects/usergems-test/venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    1722.000000
mean             inf
std              NaN
min         1.385650
25%         8.929793
50%        20.265694
75%        54.344181
max              inf
Name: md2html_ratio, dtype: float64

In [144]:
sub1 = df[df.md_length > 20000]

In [145]:
len(sub1)

81

# Website Classifications

GPT pipelines

- Check if it's right website (i.e. not just cookie banner, or can't load page, etc.)
- Contains job classification
- Extract links
- Format job description

In [3]:
import pandas as pd
from jobsfinder.gpts import *
from jobsfinder.core import cost_so_far

In [14]:
df = pd.read_csv("../data/03_valid_website.csv")

In [15]:
df.head()

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website
0,Sawa Impact Labs,https://sawa.gallery/,Success,Success,Squarespace \- Website Expired\n\n\n \nWebsite...,6998,124,valid
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,Success,OpenMI \- University of Michigan Maize Pages\n...,39237,784,valid
2,Nifty's,http://niftys.com,Success,Success,DNS points to prohibited IP \| niftys.com \| C...,5755,982,valid
3,Basewell,https://basewell.com,Success,Success,Basewell[About](/about)[Blog](/blog)[Sign in](...,322761,3504,valid
4,Journey,http://journey.io,Success,Success,"Journey – engage, close, and onboard customers...",218666,3466,valid


In [16]:
invalids = df[df.valid_website == 'invalid']

In [11]:
df.valid_website.value_counts()

valid_website
valid      1812
invalid     188
Name: count, dtype: int64

In [12]:
df = df[df.valid_website == 'valid']

In [13]:
df.head()

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website
0,Sawa Impact Labs,https://sawa.gallery/,Success,Success,Squarespace \- Website Expired\n\n\n \nWebsite...,6998,124,valid
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,Success,OpenMI \- University of Michigan Maize Pages\n...,39237,784,valid
2,Nifty's,http://niftys.com,Success,Success,DNS points to prohibited IP \| niftys.com \| C...,5755,982,valid
3,Basewell,https://basewell.com,Success,Success,Basewell[About](/about)[Blog](/blog)[Sign in](...,322761,3504,valid
4,Journey,http://journey.io,Success,Success,"Journey – engage, close, and onboard customers...",218666,3466,valid


In [ ]:
in

In [8]:
async def jobs_status(content) -> WebsiteClassification:
    _system_msg = """

You are a website classifier. I'm going to give you access to a website content (converted to markdown). Your job is to determine whether the website contains jobs.

You shall classify websites into the following buckets:
- Job list: the website explicitly lists actual jobs (with titles), or links to jobs that you can apply for
- Job open apply: the website says their accepting jobs, but does not list the jobs. Instead, it encourages people to apply, or email HR, or just has a rolling generic process.
- Link to jobs: the page does not contain job lists, but has a link to a career / job page that has it. The link could be on the same domain, a separate domain, or it could be a link to an ATS provider or job board
- No jobs: the website does not have any explicit jobs, does not have a vague apply by sending HR your CV, and does not even include a link to a separate page for jobs.

For the output, give your very short reasoning (max 1-2 sentence), plus your classification.

""".strip()

    return await simple_gpt(_system_msg, content, WebsiteClassification)